# chapter 9. Recommendation System

## Content Based Filtering - TMDB 5000 Movie DataSet

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링

콘텐츠 기반 필터링은 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성 / 속성, 구성 요소 등을 가진 다른 영화를 추천하는 것이다. <br>
**유사성을 판단하는 기준이 상품을 구성하는 다양한 콘텐츠를 기반으로 하는 방식**이 **콘텐츠 기반 필터링**이다. 

**데이터 로딩 및 가공**

In [9]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


콘텐츠 기반 필터링 추천 분석에 사용할 주요 Column만 추출해 새로 DataFrame으로 만든다. 

In [10]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

'genres', 'keywords'와 같은 Column을 보면 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열로 표기되어 있다. <br>
Column이 DataFrame으로 만들어질 때는 단순히 문자열 형태로 로딩되므로 이 Column을 가공하지 않고는 필요한 정보를 추출할 수 없다. 

In [11]:
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [12]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


'genres' Column의 문자열을 분해해서 새별 장르를 파이썬 리스트 객체로 추출한다. <br>

**텍스트 문자 1차 가공, 파이썬 딕셔너리 변환 후 리스트 형태로 변환**

In [13]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [14]:
movies_df['genres'].head(1)

0    [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
Name: genres, dtype: object

이제 'genres' Column은 문자열이 아니라 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가진다. <br>
'genres' Column에서 같은 장르명만 리스트 객체로 추출한다. <br>
apply(lambda x : [y['name'] for y in x])와 같이 변환하면 리스트 내 여러 개의 딕셔너리의 'name' 키에 해당하는 값을 찾아 이를 리스트 객체로 변환한다. 

In [15]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 콘텐츠 유사도 측정

**장르 유사도를 측정**하기 위해 **'genres'를 문자열로 변경한 뒤 이를 CountVectorizer로 Feature Vetorize한 행렬 데이터 값을 코사인 유도로 비교**한다.
1. 문자열로 변환된 'genres' Column을 Count 기반 Feature Vectorize 변환한다. 
2. 'genres' 문자열을 Feature Vectorize 행렬로 변환한 데이터 세트를 코사인 유도를 통해 비교한다. 이를 위해 데이터 세트의 레코드 별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성한다.
3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다. 

**장르 문자열의 Count 기반 Feature Vectorize**

In [20]:
type(('*').join(['test', 'test2']))

str

In [27]:
# Error ! 
# InvalidParameterError: The 'min_df' parameter of CountVectorizer must be a float in the range [0.0, 1.0] or an int in the range [1, inf). Got 0 instead.
# min_df 파라미터는 최소 문서 빈도수를 나타내며, 특정 단어의 등장 횟수가 이 값보다 작으면 무시하도록 하는 역할한다. 
# 기존 코드에서는 min_df = 0로 설정되어 있다. 그러나 min_df는 0보다 큰 값을 가져야 한다. 
# min_df 값을 0 이상의 적절한 값으로 설정하도록 한다. 

from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df = 1, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


생성된 Feature Vector 행렬에 사이킷런의 consine_simlarity()를 이용해 코사인 유사도를 계산한다. <br>
consine_simlarity()는 기준 행과 비교 행의 코사인 유사도를 행렬 형태로 반환하는 함수이다. 

<img src = './image/Consine Simlarity.jpg' style = 'width : 800px; height : 500px;'> 

Feature Vectorize 된 행렬에 consine_simlarity()를 적용한다. 

**장르에 따른 영화별 코사인 유사도 추출**

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


consine_simlarity() 호출로 생성된 'genres_sim' 객체는 'movies_df'의 'genre_literal' Column을 Feature Vectorizer한 행렬('genre_mat') 데이터의 행별 유사도 정보를 가지고 있으며, 결국은 'movies_df' DataFrame의 행별 장르 유사도 값을 가지고 있는 것이다. <br>
'movies_df'의 개별 레코드에 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출하기 위해 'genre_sim' 객체를 이용한다. <br>
'genre_sim' 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값을 추출하면 된다. 

In [29]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


'genre_sim_sorted_ind' 객체는 각 레코드의 장르 코사인 유사도가 가장 높은 순으로 정렬된 타 레코드의 위치 인덱스 값을 가지고 있다. 

### 장르 콘텐츠 필터링을 이용한 영화 추천 
장르 유사도에 따라 영화를 추천하는 함수를 생성한다. 

In [30]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고,
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, : (top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터이다.  
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [31]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


좀 더 많은 후보군을 선정한 뒤에 영의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경한다. <br>
이때 소수의 관객이 특정 영화에 만점이나 매우 높은 평점을 부여애 왜곡된 데이터를 가지고 있다. 이를 확인하기 위해 평점이 높은 영화 정보를 확인한다. <br>

In [33]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending = False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


왜곡된 평점 데이터를 회피할 수 있도록 평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요하다. <br>
평가 횟수에 대한 가중치가 부여된 평점 방식을 사용할 것이다. 

**가중 평점(weighted rating) = (v / (v + m)) * R + (m / (v + m)) * C**

- v : 개별 영화에 평점을 투표한 횟수
- m : 평점을 부여하기 위한 최소 투표 횟수
- R : 개별 영화에 대한 평균 평점
- C : 전체 영화에 대한 평균 평점

m 값을 높이면 평점 투표 횟소가 많은 영화에 더 많은 가중 평점을 부여한다. 

In [34]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C,3), 'm:', round(m, 3))

C: 6.092 m: 370.2


기존 평점을 새로운 가중 평점으로 변경하는 함수를 생성하고, 이를 이용해 새로운 평점 정보인 'vote_weighted' 값을 만든다. 

In [35]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v / (v + m)) * R) + ((m / (m + v)) * C)   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis = 1) 

In [36]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


새롭게 정의된 평점 기준에 따라 영화를 추천한다. <br>
장르 유사성이 높은 영화를 'top_n'의 2배수만큼 후보군으로 선정한 뒤에 'weighted_vote' Column 값이 높은 순으로 'top_n'만큼 추출하는 방식으로 'find_sim_movie()' 함수를 변경한다. <br>
변경된 'find_sim_movie()'를 이용해 다시 한번 유사한 영화를 콘텐츠 기반 필터링 방식으로 추천한다. 

In [37]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
